In [1]:
import os

In [2]:
%pwd

'/Users/armandoreyesmartinez/00_End_to_End_ML/Chicken_Disease_Classification_Project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/armandoreyesmartinez/00_End_to_End_ML/Chicken_Disease_Classification_Project'

In [6]:
## Preparing the entity

from dataclasses import dataclass
from pathlib import Path    

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


In [5]:
## This is a unit test that need to be improved
import unittest

# Compatibility for different Python versions
if not hasattr(unittest.TestCase, 'assertRaisesRegexp'):
    unittest.TestCase.assertRaisesRegexp = unittest.TestCase.assertRaisesRegex


from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config
        

In [8]:
## Libraries for component
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

2025-08-24 22:43:44.651337: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [11]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2025-08-24 22:45:31,600 : INFO : common  : created directory at: artifacts ]
[2025-08-24 22:45:31,602 : INFO : common  : created directory at: artifacts/prepare_callbacks/checkpoint_dir ]
[2025-08-24 22:45:31,603 : INFO : common  : created directory at: artifacts/prepare_callbacks/tensorboard_log_dir ]


In [12]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2025-08-24-22-47-55'